# Установка библиотек

In [1]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [2]:
!pip install scipy

In [3]:
!pip install statsmodels


# Загрузка данных

Для скачивания данных используем команду `!wget`. 

Данные появятся в локальной директории.

Мы будем использовать информацию о диаметре пицц разных производителей.

In [4]:
!wget https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv

--2022-03-09 17:22:56--  https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 200 OK
Длина: 539 [text/plain]
Сохранение в каталог: ««pizzas.csv»».

pizzas.csv          100%[===================>]     539  --.-KB/s    за 0s      

2022-03-09 17:22:57 (27,5 MB/s) - «pizzas.csv» сохранён [539/539]



Загрузим данные в память ноутбука.

In [5]:
data = pd.read_csv('pizzas.csv')

In [6]:
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)

In [7]:
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


# Проверка данных на нормальность


In [8]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [9]:
alpha = 0.05

## Тест Шапиро-Уилка

In [10]:
from scipy.stats import shapiro

In [11]:
_, p = shapiro(data)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


## Тест Д’Агостино

In [12]:
from scipy.stats import normaltest

In [13]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально


# Независимый T-тест 

In [14]:
from scipy.stats import ttest_ind

In [15]:
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

Зависимая переменная (диаметр пиццы) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [16]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


-----

# Корреляция Спирмана 

In [17]:
from numpy.random import rand
from scipy.stats import spearmanr

data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)


corr, p = spearmanr(data1, data2)

In [18]:
print(corr,p)

0.8979322299322299 0.0


In [19]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


# ANOVA тест

In [20]:
from scipy.stats import f_oneway

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [21]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [22]:
_, p = f_oneway(petersburg, magadan, tvarminne)

In [23]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в трех разных местах.'

In [24]:
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143829 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в трех разных местах.


_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [25]:
import pandas as pd
from scipy.stats import shapiro     # Критерий Shapiro-Wilk
from scipy.stats import ttest_ind   # Независимый t-тест

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

**Создание DataFrame**

In [26]:
# Создание DataFrame
mussels = pd.DataFrame(
    data={
        'petersburg': pd.Series(petersburg, index=range(7)),
        'magadan': pd.Series(magadan, index=range(8))
    }
)

# Заполняем пропущенное значение медианой
mussels.fillna({'petersburg': mussels['petersburg'].median()}, inplace=True)

display(mussels)

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


**Проверка признаков на нормальность**

* *Нулевая гипотеза H0: размеры раковин распределены по нормальному закону*
* *Альтернативная гипотеза Ha: распределение размеров раковин не является нормальным*

*Принимаем стандартное значение уровня значимости alpha = 0.05*

*Для проверки признаков на соответствие нормальному распределению применим критерий Шапиро-Уилка*

In [27]:
_, p = shapiro(mussels)
print(f"Критерий Шапиро-Уилка: p-value = {p: .3f}")

Критерий Шапиро-Уилка: p-value =  0.101


*Вывод: p-value больше принятого уровня значимости 0.05, следовательно нулевая гипотеза не отвергается, а распределение размеров раковин можно считать нормальным*

**Проверка на наличие корреляции**

*Поскольку данные распределены по нормальному закону, следует применять параметрические тесты, в случае проверки на корреляцию непрерывных числовых признаков- это коэффициент корреляции r Пирсона*

In [28]:
display(mussels.corr(method='pearson'))

,petersburg,magadan
petersburg,1.000000,0.256665
magadan,0.256665,1.000000


*Вывод: коэффициент корреляции 0.26 свидетельствует об отсутствии связи между признаками либо об очень слабой связи*

**Сравнение средних размеров по выборкам**

* *Нулевая гипотеза H0: средние размеры раковин в двух выборках (Петербург и Магадан) не отличаются*
* *Альтернативная гипотеза Ha: есть статистически значимое различие в средних размерах раковин*

*Принимаем стандартное значение уровня значимости alpha = 0.05*

*Поскольку признаки распределены нормально (необходим параметрический тест), сравниваются только две выборки по одному параметру (не нужен дисперсионный анализ), объем выборок не превышает 30 (исключаем z-тест), и выборки независимы (происходят из разных совокупностей), применяем t-test (критерий Стьюдента) для независимых выборок. Предполагаем равенство дисперсий в двух выборках.*

In [29]:
_, p = ttest_ind(mussels['petersburg'], mussels['magadan'])
print(f'Независимый t-тест: p-value = {p: .3f}')

Независимый t-тест: p-value =  0.003


*p-value меньше установленного уровня значимости 0.05 (более того, меньше уровня 1%), следовательно есть статистические значимые различия в рамерах раковин. Рассчитаем средние размеры:*

In [30]:
aver_peter = mussels['petersburg'].mean()
aver_magadan = mussels['magadan'].mean()
print(f'Средний размер раковины (Петербург): {aver_peter}')
print(f'Средний размер раковины (Магадан): {aver_magadan}')

Средний размер раковины (Петербург): 0.1032125
Средний размер раковины (Магадан): 0.0780125


*Вывод: мидии из Петербурга в среднем крупнее, различия статистически значимы*

# Бонус

Скачаем датасет, содержащий информацию по давлению 120 человек.



In [31]:
!wget https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv

--2022-03-09 17:24:04--  https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 200 OK
Длина: 2851 (2,8K) [text/plain]
Сохранение в каталог: ««blood_pressure.csv»».

blood_pressure.csv  100%[===================>]   2,78K  --.-KB/s    за 0s      

2022-03-09 17:24:05 (46,4 MB/s) - «blood_pressure.csv» сохранён [2851/2851]



In [32]:
data = pd.read_csv('blood_pressure.csv')

In [33]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


In [34]:
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


## Z-тест

Проведем Z-тест для проверки двух независимых групп данных на равенство средних значений выборок.

In [35]:
from statsmodels.stats import weightstats

In [36]:
_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))


if p < 0.05:
    print("Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датачете равна 40%.

$p_0$: гипотетическая доля мужчин = 0.40

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [37]:
p_0 = 0.4
n = len(data)
x = len(data[data.sex == 'Male'])

print(n, x)

120 60


In [38]:
from statsmodels.stats.proportion import proportions_ztest

#perform one proportion z-test
_, p = proportions_ztest(count=x, nobs=n, value=p_0)

In [39]:
if p < 0.05:
    print("Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу.")

Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.
